In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from lib.data_utils import *
from lib.model_utils import *

from lib.simulator import *

from glob import glob

In [2]:
def sim_models(model_name):
    sim_basemodel = Simulator(
        model_type_path=model_name.replace('ACC', 'TYPE'),
        model_acc_path=model_name,
        model_data_path=model_name.replace('ACC', 'DATA'),
        device='cuda:0'
    )

    feature_tensor = sim_basemodel.simulate([1 if i == 27 else 0 for i in range(33)] + [0, 0, 0.5, 0.5, 1, 1, 0, 0, 0], store_full_sim=False, n_sims=3000, disable_tqdm=True)

    data = pd.DataFrame(feature_tensor.cpu())
    data['type_id'] = data[data.columns[:33]].idxmax(axis=1) + 1
    data = data.drop(data.columns[:33], axis=1)
    data.columns = ['period', 'minute', 'x', 'y', 'is_home_team', 'accurate', 'goal', 'home_score', 'away_score', 'type_id']
    data.home_score = (data.home_score * 10).astype(int)
    data.away_score = (data.away_score * 10).astype(int)

    return [
        (data.home_score > data.away_score).mean() * 3 + (data.home_score == data.away_score).mean(),
        (data.home_score < data.away_score).mean() * 3 + (data.home_score == data.away_score).mean(),
        data.home_score.mean(),
        data.away_score.mean(),
        ]

# Simulations

In [3]:
sim_data = []
# for fname in tqdm(glob(f'models/finetuning/*/*/*.pth')):
for fname in tqdm(glob(f'models/finetuning/player_adding_2/*/*.pth')):
    if 'ACC' not in fname:
        continue
    model_type = fname.split('/')[2]
    model_name = fname.split('/')[-2]
    model_id = fname.split('/')[-1].replace('_ACC.pth', '')
    sim_data.append([model_type, model_name, model_id] + sim_models(fname))

  0%|          | 0/1200 [00:00<?, ?it/s]

100%|██████████| 1200/1200 [31:30<00:00,  1.58s/it]


In [4]:
for i in range(10):
    sim_data.append(['base', 'base', str(i)] + sim_models('models/lem/LEMv4_MODEL_ACC_TORCH.pth'))

# Saving Data

In [ ]:
pd.DataFrame(sim_data, columns=['model_type', 'model_name', 'model_id', 'home_points', 'away_points', 'home_goals', 'away_goals']).to_csv('res/sim_finetuning_results_v11.csv')